### Requirements:

You need to .....

1. simualtion track
2. hardware track
    - DB21M up and running
    - 2 AprilTags

### Intended outcomes:

Calibrate the Duckiebot kinematic model using the data from the wheel encoders.

### Steps:

1. Theory  
2. Approach
3. Implementation
4. Validation



1. Theory -> Review odometry class
2. Approach -> 
    - Use wheel encoders to estimate the pose of the DB using deadrecknonig approximation. 
    - Use estimate and ground truth to best fit

3. Implementation ->
    - Read the data from the wheel encoders 
        - Sim:
        - HW :
            
    - Obtain the gorund truth from AprilTags


4. Validation

1. Theoretical understanding of the Duckiebot kinematic model, in particular what the parameters _R_ and _L_ represent.
2. Approach:
    - Wheel encoders and deadrecknonig
    
    - Straight path 
    - Curved path (e.g., sinusoidal)

3. Validation of the resulting parameters.



In [2]:
import numpy as np

# 🚙 💻 Step 1: obtain wheel encoder measurements. 

The first step to evaluate to Duckiebot's odometry is to measure the data from the wheel encoders. Let's get some of these messages and understand what they mean.

## 💻 Read data from wheel encoders

- Open a terminal on your computer 
- `dts exercises build` 
-  `dts exercises test --sim`
    - find localhost address novnc and paste in browser
    - open RQT image view, select compressed image topic from the dropdown menu
    - open LX terminal inside novnc
    - rostopic list
    - rostopic echo /agent/left_wheel_encoder_node/tick (right wheel)
       - you should see no data
    - open joypad / virtual joystick on novnc desktop
    - press arrows
    - you should see: images moving (don't crash on tree or you will have to restart!)
    - on terminal, see wheel encoder data, e.g.
    
```    
---
header: 
  seq: 527
  stamp: 
    secs: 0
    nsecs:         0
  frame_id: ''
data: 82
resolution: 135
type: 1
---
```    

How to read the wheel encoder data: 

* `seq`: is ...
* `data`: is the cumulative count of ticks from the wheel so far
* note that ticks increase or decrease depending on if you are going forward or backwards
* `resolution`: is the total number of ticks per encoders
* `type`: ?

## 🚙 Read data from wheel encoders

- Open a terminal on your computer
- - `dts exercises build` (unless you did it already, but it won't break anything if you do it again)
- Make sure your Duckiebot is on and connected to the networks
    - You can test this by opening the Dashboard or `ping ROBOTNAME.local` from your computer terminal
-  `dts exercises test -b ROBOTNAME`
    - find localhost address novnc and paste in browser (e.g. localhost:8087)
    - open RQT image view, select compressed image topic from the dropdown menu
        - you should see what your robot sees
    - open LX terminal inside novnc
        - rostopic list
        - rostopic echo /agent/left_wheel_encoder_node/tick (right wheel)
        - you should see no data
    - open joypad / virtual joystick on novnc desktop (or just move wheels by hand)
    - press arrows
    - you should see: robot moving and data streaming
    - on terminal, see wheel encoder data, e.g.

``` 
---
header: 
  seq: 5594
  stamp: 
    secs: 0
    nsecs:         0
  frame_id: ''
data: 13
resolution: 135
type: 1
---
```

## Reading the number of ticks from each wheel

The wheel encoder message above provides several pieces of information. Let's extract the number of ticks. 


In [ ]:
# write a function to get value of "data" from each wheel
fn = ...

## 1. Evaluating rotation of each wheel

Let's use the above to measure how much each wheel has rotated 

### Wheel encoder calibration factor

- There are 135 ticks per revolution on these wheel encoders. 

In [3]:
N_tot = 135 # total number of ticks per revolution
alpha = 2 * np.pi / N_tot # wheel rotation per tick

Call fn defined above:

- At time t: ticks_l(t) = measure ticks from one wheel
- At time t+1: ticks_l(t+1) = measure ticks from same wheel
- delta_ticks_left = ticks_l(t+1) - ticks_l(t)

In [ ]:
# write function to get delta ticks

delta_ticks_left = ... # delta ticks of left wheel in arbitrary dt
delta_ticks_right = ... # delta ticks of right wheel in same dt

rotation_wheel_left = alpha * delta_ticks_left # total rotation of left wheel in dt
rotation_wheel_right = alpha * delta_ticks_right # total rotation of right wheel in dt

## 2. Evaluate distance travelled by each wheel

Now let's calculate the distance travelled by each wheel. It depends on the wheel radii. 

Take a ruler and measure you wheel radii (let's assume they are the same):

In [ ]:
R = 0.033 # insert value measured by ruler, in *meters* (Jacopo measures a diameter of 6.6cm)

In [ ]:
# Distance travelled by each wheel in one time step: 

d_left = R * delta_ticks_left 
d_right = R * delta_ticks_left

## 3. Evaluate distance travelled by the origin of the robot frame, and the robot's rotation

The travelled distance of point A (origin of the robot frame) is given by the average of the distances travelled by the wheels (given they symmetry assumption of the Duckiebot):

In [ ]:
d_A = (d_left + d_right)/2

### Step 10

Write in the cell below the code to estimate the pose of the Duckiebot using data given by the wheel encoders.

**DO NOT CHANGE THE NAME OF THE FUNCTION**

In [1]:

# import numpy as np # already imported above 

# DO NOT CHANGE THE NAME OF THIS FUNCTION
def poseEstimation( R,
                    L,
                    x_prev,
                    y_prev,
                    theta_prev,
                    delta_phi_left,
                    delta_phi_right):
    """
        Calculate the current Duckiebot pose using dead reckoning approach,
        based on the kinematic model.

        Returns:
            x_curr, y_curr, theta_curr (:double: values)
    """
    # x_curr = x_prev + R*(delta_phi_left+delta_phi_right)*np.cos(theta_prev)/2
    # y_curr = y_prev + R*(delta_phi_left+delta_phi_right)*np.sin(theta_prev)/2
    # theta_curr = theta_prev + R*(delta_phi_right-delta_phi_left)/(2*L)
        
    
    w = [R, R/L, 1]
    x = np.array(
        [
            [
                (delta_phi_left+delta_phi_right)*np.cos(theta_prev)/2,
                (delta_phi_left+delta_phi_right)*np.sin(theta_prev)/2,
                0
            ],
            [
                0,
                0,
                (delta_phi_right-delta_phi_left)/2],
            [
                x_prev,
                y_prev,
                theta_prev
            ]
        ])

    x_curr, y_curr, theta_curr = np.array(w).dot(x)
    
    return x_curr, y_curr, theta_curr

### Unit Test

Let's see if the function you wrote above passes the following test!

Unit tests are useful to check if the piece of code you write does its job correctly.
If the tests are good you will not see any output.


In [2]:
assert((poseEstimation(0.0318, 0.08, 0, 0, 0, -10*np.pi/180, 30*np.pi/180) == np.array([0.005550147021341967, 0.0, 0.1387536755335492])).all())
assert((poseEstimation(0.0318, 0.08, 0, 0, 30*np.pi/180, 30*np.pi/180, 45*np.pi/180) == np.array([0.01802463118207754, 0.010406525665016188, 0.5756314039233797])).all())
assert((poseEstimation(0.0318, 0.08, 0, 0, 0, 45*np.pi/180, 30*np.pi/180) == np.array([0.02081305133003238, 0.0, -0.05203262832508096])).all())

### Build the Activity

In [ ]:
! cd .. && dts exercises build

### Run the Activity in simulation

In [ ]:
! cd .. && dts exercises test --sim

After running the above command, open the following link http://localhost:8087 you will visualize the VNC. Then from the VNC open RVIZ, press the `Add` on the bottom left side of the window, after that go to the tab `topics` and select `/DUCKIEBOT_NAME/encoder_localization`. Finally open the joystick (on the Desktop) and move the robot. You will see that the pose arrow start moving according to the given commands.

### Run the activity on the Duckiebot

In [ ]:
! cd .. && dts exercises test --duckiebot_name ![DB_NAME] 

After running the above command, open the following link http://localhost:8087 you will visualize the VNC. Then from the VNC open RVIZ, press the `Add` on the bottom left side of the window, after that go to the tab `topics` and select `/DUCKIEBOT_NAME/encoder_localization`. Finally open the joystick (on the Desktop) and move the robot. You will see that the pose arrow start moving according to the given commands.